In [1]:
def groups(lists):
    #求两个数组的交集，并集，差集
    group = []
    intersection = [i for i in lists[0] if i in lists[1]]
    intersection = list(set(lists[0]).intersection(set(lists[1])))
    group.append(str(intersection))
    union = list(set(lists[0]).union(set(lists[1])))
    group.append(str(union))
    subtraction = list(set(lists[1]).difference(set(lists[0])))
    group.append(str(subtraction))
    return group

In [2]:
from clipper_admin import ClipperConnection, DockerContainerManager                                                                                                                                     
from clipper_admin.deployers.python import deploy_python_closure   

In [3]:
clipper_conn = ClipperConnection(DockerContainerManager())
clipper_conn.start_clipper()

20-03-09:00:28:55 INFO     [docker_container_manager.py:184] [default-cluster] Starting managed Redis instance in Docker
20-03-09:00:28:59 INFO     [docker_container_manager.py:276] [default-cluster] Metric Configuration Saved at /tmp/tmp2h82sv43.yml
20-03-09:00:29:00 INFO     [clipper_admin.py:162] [default-cluster] Clipper is running


In [4]:
from clipper_admin.deployers import python as python_deployer

In [5]:
python_deployer.deploy_python_closure(clipper_conn, name="group-model", version=1, input_type="int", func=groups)

20-03-09:00:29:12 INFO     [deployer_utils.py:41] Saving function to /tmp/tmp8676060bclipper
20-03-09:00:29:12 INFO     [deployer_utils.py:51] Serialized and supplied predict function
20-03-09:00:29:12 INFO     [python.py:192] Python closure saved
20-03-09:00:29:12 INFO     [python.py:206] Using Python 3.6 base image
20-03-09:00:29:12 INFO     [clipper_admin.py:534] [default-cluster] Building model Docker image with model data from /tmp/tmp8676060bclipper
20-03-09:00:29:13 INFO     [clipper_admin.py:539] [default-cluster] Step 1/2 : FROM clipper/python36-closure-container:0.4.1
20-03-09:00:29:13 INFO     [clipper_admin.py:539] [default-cluster]  ---> e5b9dc250c05
20-03-09:00:29:13 INFO     [clipper_admin.py:539] [default-cluster] Step 2/2 : COPY /tmp/tmp8676060bclipper /model/
20-03-09:00:29:13 INFO     [clipper_admin.py:539] [default-cluster]  ---> 23a132842686
20-03-09:00:29:13 INFO     [clipper_admin.py:539] [default-cluster] Successfully built 23a132842686
20-03-09:00:29:13 INFO   

In [6]:
clipper_conn.register_application(name="group-world", input_type="int", default_output="-1.0", slo_micros=100000)

20-03-09:00:29:31 INFO     [clipper_admin.py:236] [default-cluster] Application group-world was successfully registered


In [7]:
clipper_conn.link_model_to_app(app_name="group-world", model_name="group-model")

20-03-09:00:29:34 INFO     [clipper_admin.py:303] [default-cluster] Model group-model is now linked to application group-world


In [11]:
def predict_false(addr, x, batch=False):
    url = "http://%s/group-world/predict" % addr

    if batch:
        req_json = json.dumps({'input_batch': x})
    else:
        req_json = json.dumps({'input': list(x)})

    headers = {'Content-type': 'application/json'}
    start = datetime.now()
    r = requests.post(url, headers=headers, data=req_json)
    end = datetime.now()
    latency = (end - start).total_seconds() * 1000.0
    print("'%s', %f ms" % (r.text, latency))

In [8]:
def predict(addr, x, batch=True):
    url = "http://%s/group-world/predict" % addr

    if batch:
        req_json = json.dumps({'input_batch': x})
    else:
        req_json = json.dumps({'input': list(x)})

    headers = {'Content-type': 'application/json'}
    start = datetime.now()
    r = requests.post(url, headers=headers, data=req_json)
    end = datetime.now()
    latency = (end - start).total_seconds() * 1000.0
    print("'%s', %f ms" % (r.text, latency))

In [10]:
import json ,requests
from datetime import datetime
addr = "127.0.0.1:1337"
# x = '{"input": [7,2,3]}'
x = [1,8,2]
predict(addr,x,batch=True)

'{"error":"Json error","cause":"Error parsing JSON: Array input of type Number is not of type array. Expected JSON schema: \n  {\n   \"input\" := [double] | [int] | [string] | [byte] | [float],\n  }\n"}', 7.094000 ms


In [14]:
import json ,requests
from datetime import datetime
addr = "127.0.0.1:1337"
# x = '{"input": [7,2,3]}'
x = [1,8,2]
x1 = [[1,2,3],[2,3,4,5]]
predict(addr,x1,batch=False)

'{"error":"Json error","cause":"Error parsing JSON: Array input of type Array is not of type int. Expected JSON schema: \n  {\n   \"input\" := [double] | [int] | [string] | [byte] | [float],\n  }\n"}', 6.979000 ms


In [15]:
import requests, json, numpy as np
headers = {"Content-type": "application/json"}
requests.post("http://localhost:1337/group-world/predict", headers=headers, data=json.dumps({"input": [1,2,3,4,5,6][2,3]})).json()

TypeError: list indices must be integers or slices, not tuple

In [1]:
clipper_conn.stop_all()

NameError: name 'clipper_conn' is not defined

怎么 input 两个元素？？